<a href="https://colab.research.google.com/github/rboghe/cened/blob/master/cened_2_0_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [86]:
import os
import urllib.request
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
from lightgbm import LGBMRegressor

# Silence pandas setting with copy warning
pd.options.mode.chained_assignment = None

# Load data

In [87]:
# Cened 2.0
url = 'https://www.dati.lombardia.it/api/views/bbky-sde5/rows.csv?accessType=DOWNLOAD'

if os.path.isdir('/tmp'):
    if os.path.isfile('/tmp/cened20.csv'):
      pass
    else:
      urllib.request.urlretrieve(url, '/tmp/cened20.csv')
else:
  os.mkdir('/tmp')
  urllib.request.urlretrieve(url, '/tmp/cened20.csv')

In [88]:
# DDH
url = 'https://raw.githubusercontent.com/rboghe/cened/master/degreedays.txt'

urllib.request.urlretrieve(url, '/tmp/ddh.csv')

ddh = pd.read_csv('/tmp/ddh.csv', usecols = ['comune','dd'])

In [89]:
cols = ['INTERO_EDIFICIO', 'RIQUALIFICAZIONE_ENERGETICA', 'ZONA_CLIMATICA',
           'ANNO_COSTRUZIONE','SUPERF_UTILE_RISCALDATA', 'SUPERF_UTILE_RAFFRESCATA',
           'VOLUME_LORDO_RISCALDATO', 'VOLUME_LORDO_RAFFRESCATO', 'CLASSE_ENERGETICA', 
           'SUPERFICIE_DISPERDENTE', 'RAPPORTO_SV','A_SOL_EST_A_SUP_UTILE','Y_IE', 
           'CLASSIFICAZIONE_DPR', 'COMUNE', 'EP_H_ND','EP_GL_NREN','EP_GL_REN', 
           'NUMERO_UNITA_IMMOBILIARI', 'RISTRUTTURAZIONE_IMPORTANTE', 'Y', 
           'CLIMATIZZAZIONE_INVERNALE', 'CI_TIPO_IMPIANTO_1', 'CI_ANNO_INSTALLAZIONE_1',
           'CI_VETTORE_ENERGETICO_1', 'CI_EFFICIENZA_MEDIA', 'CI_POTENZA_NOMINALE_2', 'CI_POTENZA_NOMINALE_1',
           'PFR_TIPO_IMPIANTO_1', 'PFR_ANNO_INSTALLAZIONE_1','PFR_VETTORE_ENERGETICO_1',
           'PFR_EFFICIENZA_MEDIA_1', 'PFR_POTENZA_NOMINALE_1', 'VENTILAZIONE_MECCANICA',
           'PFR_TIPO_IMPIANTO_2', 'PFR_ANNO_INSTALLAZIONE_2','PFR_VETTORE_ENERGETICO_2',
           'PFR_EFFICIENZA_MEDIA_2', 'PFR_POTENZA_NOMINALE_2']
  
cened_new = pd.read_csv('/tmp/cened20.csv', usecols = cols)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (161,163) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Preprocessing

In [90]:
# Fillna
for col in ['CI_TIPO_IMPIANTO_1', 'CI_ANNO_INSTALLAZIONE_1','CI_VETTORE_ENERGETICO_1',
           'CI_EFFICIENZA_MEDIA', 'CI_POTENZA_NOMINALE_1','PFR_TIPO_IMPIANTO_1', 
            'PFR_ANNO_INSTALLAZIONE_1','PFR_VETTORE_ENERGETICO_1', 'PFR_EFFICIENZA_MEDIA_1', 
            'PFR_POTENZA_NOMINALE_1', 'VENTILAZIONE_MECCANICA','PFR_TIPO_IMPIANTO_2', 
            'PFR_ANNO_INSTALLAZIONE_2','PFR_VETTORE_ENERGETICO_2', 'PFR_EFFICIENZA_MEDIA_2', 
            'PFR_POTENZA_NOMINALE_2','CI_POTENZA_NOMINALE_2','EP_GL_NREN', 'EP_GL_REN']:
    cened_new[col].fillna(0, inplace=True)

# Drop buildings with NaN values
cened_new.dropna(inplace = True)

# Only entire buildings
cened_new = cened_new[cened_new['INTERO_EDIFICIO'] == True]

# Drop renovations
#cened_new = cened_new[cened_new['RISTRUTTURAZIONE_IMPORTANTE'] == False]
#cened_new = cened_new[cened_new['RIQUALIFICAZIONE_ENERGETICA'] == False]

# We'll use residential buildings only
cened_new['CLASSIFICAZIONE_DPR'] = cened_new['CLASSIFICAZIONE_DPR'].str.replace(r'E.1 (1)', 'E.1(1)')
cened_new = cened_new[cened_new['CLASSIFICAZIONE_DPR'] == 'E.1(1)']

# Add ddh
cened_new['COMUNE'] = cened_new['COMUNE'].str.lower()
ddh['comune'] = ddh['comune'].str.lower()
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"`",  "'")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"o`",  "o'")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"baranzate",  "bollate")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"bovisio masciago",  "bovisio-masciago")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"cornate d`adda",  "cornate d'adda")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"cortenuova",  "cortenova")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"sant'omobono terme",  "sant'omobono imagna")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"borgo virgilio",  "virgilio")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r" - ",  "-")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"cassina de pecchi",  "cassina de' pecchi")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"cerano intelvi",  "cerano d'intelvi")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"colverde",  "drezzo")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"cornale e bastida",  "cornale")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"corteolona e genzone",  "corteolona")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"costa serina",  "costa di serina")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"gabbioneta binanuova",  "gabbioneta-binanuova")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"gadesco pieve delmona",  "gadesco-pieve delmona")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"gerre de'caprioli",  "gerre de' caprioli")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"gravedona ed uniti",  "grandola ed uniti")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"la valletta brianza",  "perego")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"lonato del garda",  "lonato")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"maccagno con pino e veddasca",  "maccagno")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"ramponio vernia",  "ramponio verna")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"sermide e felonica",  "sermide")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"toscolano maderno",  "toscolano-maderno")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"tremezzina",  "tremezzo")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"val brembilla",  "brembilla")
cened_new['COMUNE'] = cened_new['COMUNE'].str.replace(r"verderio",  "verderio inferiore")

ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"é",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"ò",  "o'")
ddh['comune'] = ddh['comune'].str.replace(r"ù",  "u'")

cened_new = cened_new.merge(ddh, left_on='COMUNE', right_on='comune', how='left')

# Riqualificazione
cened_new['RIQUALIFICAZIONE_ENERGETICA'] = cened_new['RIQUALIFICAZIONE_ENERGETICA'].astype("category").cat.codes

# Ristrutturazione
cened_new['RISTRUTTURAZIONE_IMPORTANTE'] = cened_new['RISTRUTTURAZIONE_IMPORTANTE'].astype("category").cat.codes

# Climatizzazione
cened_new['CLIMATIZZAZIONE_INVERNALE'] = cened_new['CLIMATIZZAZIONE_INVERNALE'].astype("category").cat.codes

# Correct floats
for col in ['SUPERF_UTILE_RISCALDATA',
           'SUPERF_UTILE_RAFFRESCATA','VOLUME_LORDO_RISCALDATO','VOLUME_LORDO_RAFFRESCATO','SUPERFICIE_DISPERDENTE',
           'RAPPORTO_SV','A_SOL_EST_A_SUP_UTILE','Y_IE','EP_H_ND','EP_GL_NREN','EP_GL_REN']:
    cened_new[col] = cened_new[col].astype(str)
    cened_new[col] = cened_new[col].str.replace(r',', '')
    cened_new[col] = cened_new[col].astype("float")

In [91]:
#EPH
cened_new['EPG'] = cened_new['EP_GL_NREN'] + cened_new['EP_GL_REN']

# Feature engineering

In [92]:
# Mean height
cened_new['ALTEZZA_MEDIA'] = cened_new['VOLUME_LORDO_RISCALDATO']/cened_new['SUPERF_UTILE_RISCALDATA']

# Period
cened_new.ANNO_COSTRUZIONE = cened_new.ANNO_COSTRUZIONE.astype(str)
cened_new.ANNO_COSTRUZIONE = cened_new.ANNO_COSTRUZIONE.map(lambda x:x[-4:])
cened_new.ANNO_COSTRUZIONE = cened_new.ANNO_COSTRUZIONE.astype(int)

cened_new['PERIODO'] = pd.cut(cened_new.ANNO_COSTRUZIONE, bins=[0,1930,1945,1960,1976,1992,2006,3000],
     labels=['Before 1930','1930-1945','1946-1960','1961-1976', '1977-1992', '1993-2006','After 2006']).cat.codes

# East facing surface
cened_new['A_SOL_EST'] = cened_new['A_SOL_EST_A_SUP_UTILE']/cened_new['SUPERF_UTILE_RISCALDATA'] #?????????

# Filters

In [93]:
cened_new = cened_new[cened_new['EP_H_ND'] < 1000]

In [94]:
cened_new = cened_new[cened_new['SUPERF_UTILE_RISCALDATA'] >= 30]

In [95]:
cened_new = cened_new[cened_new['VOLUME_LORDO_RISCALDATO'] >= 75]

In [96]:
cened_new = cened_new[cened_new['ALTEZZA_MEDIA'] > 2.5]
cened_new = cened_new[cened_new['ALTEZZA_MEDIA'] < 5]

In [97]:
cened_new = cened_new[cened_new['Y_IE'] <= 6]
cened_new = cened_new[cened_new['Y_IE'] >= 0.01] #0.03

In [98]:
#cened_new = cened_new[cened_new['CI_POTENZA_NOMINALE_1'] > 0]
#cened_new = cened_new[cened_new['CI_POTENZA_NOMINALE_1'] < 200]

In [99]:
#cened_new = cened_new[cened_new['CI_EFFICIENZA_MEDIA'] >= 0.2]
#cened_new = cened_new[cened_new['CI_EFFICIENZA_MEDIA'] <= 5]

In [100]:
cened_new = cened_new[cened_new['A_SOL_EST_A_SUP_UTILE'] <= 0.5]
cened_new = cened_new[cened_new['A_SOL_EST_A_SUP_UTILE'] > 0]

In [101]:
len(cened_new)

32901

# Shuffle

In [102]:
cened = cened_new.sample(frac=1).reset_index(drop=True)

# Reserve a test set

In [103]:
msk = np.random.rand(len(cened)) < 0.8
train = cened[msk]
test = cened[~msk]

In [104]:
print(len(train))

26255


In [105]:
print(len(test))

6646


# Define MAPE

In [106]:
def neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  return -np.abs((y_true - y_pred)/y_true).mean()

In [107]:
def modified_neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  errors = np.abs((y_true - y_pred)/y_true)
  index = np.argwhere(errors > 1)
  mod_errors = np.delete(errors, index)
  return -mod_errors.mean()

In [108]:
neg_mape_scorer = make_scorer(neg_mape, greater_is_better=True)

In [109]:
modified_neg_mape_scorer = make_scorer(modified_neg_mape, greater_is_better=True)

# Random search

In [141]:
mlcol = ['SUPERF_UTILE_RISCALDATA',
           'VOLUME_LORDO_RISCALDATO',
           'SUPERFICIE_DISPERDENTE', 'dd','PERIODO',
           'RAPPORTO_SV','A_SOL_EST_A_SUP_UTILE','Y_IE',
           'ALTEZZA_MEDIA', 'RIQUALIFICAZIONE_ENERGETICA','RISTRUTTURAZIONE_IMPORTANTE',
             'A_SOL_EST', 'NUMERO_UNITA_IMMOBILIARI']

In [142]:
cat_feats = ['PERIODO']

In [143]:
param_dist = {'feature_fraction': np.linspace(0.4, 1, num=7),
              'num_leaf' : list(range(20,40)),
             'max_depth' : [-1],
             'max_bin' : [100, 200, 300, 500, 750, 1000, 2000],
             'bagging_fraction' : np.linspace(0.4, 1, num=14),
             'bagging_freq' : list(range(1,10)),
             'lambda_l1' : stats.uniform(0, 0.6),
             'lambda_l2' : stats.uniform(0, 0.6)}

lgbm = LGBMRegressor(n_estimators = 100, silent = True, verbose = 0, is_training_metric = True, n_jobs = 1, 
                     eval_metric  = 'mape')

n_iter_search = 100

random_search = RandomizedSearchCV(lgbm, param_distributions=param_dist, n_iter=n_iter_search, 
                                   scoring=modified_neg_mape_scorer, cv = 5, n_jobs = 6, verbose = 2)

random_search.fit(cened[mlcol], cened['EP_H_ND'], categorical_feature = cat_feats)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   13.3s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:  1.2min
[Parallel(n_jobs=6)]: Done 353 tasks      | elapsed:  2.8min
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:  3.9min finished
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['PERIODO']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           eval_metric='mape',
                                           importance_type='split',
                                           is_training_metric=True,
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=1, num_leaves=31,
                                           objective=None, random_sta...
                                        'lambda_l1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0a773c4828>,
            

# Print results

In [144]:
print(random_search.best_params_)

{'bagging_fraction': 0.8615384615384616, 'bagging_freq': 9, 'feature_fraction': 1.0, 'lambda_l1': 0.05869879136219036, 'lambda_l2': 0.5647457425082241, 'max_bin': 300, 'max_depth': -1, 'num_leaf': 39}


In [145]:
print(random_search.best_score_)

-0.2348192513866231


In [146]:
# Create a results df
cv_df = pd.DataFrame(random_search.cv_results_)
cv_df = cv_df.sort_values(by = ['rank_test_score']).reset_index()

# Filter columns
res_cols = [col for col in cv_df if col.startswith('split')]

# Select best iter
best_res = cv_df.loc[0]

# Print results for each fold
print(best_res[res_cols])

split0_test_score   -0.236666
split1_test_score   -0.234864
split2_test_score   -0.231509
split3_test_score   -0.236418
split4_test_score   -0.234639
Name: 0, dtype: object


# Train the final model

In [147]:
light = LGBMRegressor(n_estimators = 1000, silent = False, verbose = 2, is_training_metric = True, n_jobs = 6)
light.set_params(**random_search.best_params_)

LGBMRegressor(bagging_fraction=0.8615384615384616, bagging_freq=9,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=1.0, importance_type='split',
              is_training_metric=True, lambda_l1=0.05869879136219036,
              lambda_l2=0.5647457425082241, learning_rate=0.1, max_bin=300,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=6, num_leaf=39,
              num_leaves=31, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=False, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=2)

In [148]:
light.fit(train[mlcol], train['EP_H_ND'])

LGBMRegressor(bagging_fraction=0.8615384615384616, bagging_freq=9,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=1.0, importance_type='split',
              is_training_metric=True, lambda_l1=0.05869879136219036,
              lambda_l2=0.5647457425082241, learning_rate=0.1, max_bin=300,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=6, num_leaf=39,
              num_leaves=31, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=False, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=2)

# Test the model

In [149]:
y_pred = light.predict(test[mlcol])

In [150]:
results_df = pd.DataFrame({'predicted' : y_pred, 'true' : test['EP_H_ND']})
results_df['error'] = np.abs((results_df['true'] - results_df['predicted'])/results_df['true'])*100

In [151]:
# MAPE
results_df['error'].mean()

32.396839006945335

In [152]:
# MAPE without >100%
results_df[results_df.error <= 100].error.mean()

23.724833963995568

In [153]:
results_df

,predicted,true,error
4,258.732389,350.57,26.196654
7,198.385245,227.31,12.724806
13,333.960173,338.39,1.309089
28,130.815895,94.08,39.047507
30,236.791823,200.16,18.301271
...,...,...,...
32889,275.902577,349.70,21.103066
32891,91.553891,54.75,67.221719
32893,312.186444,399.14,21.785227
32896,82.225885,28.55,188.006604
